# Parsing all the Data from the Files available on "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University"  

Link : https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

In [315]:
import pandas as pd
import os
import ast
import numpy as np

In [37]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\csse_covid_19_data\\csse_covid_19_daily_reports\\"

In [38]:
df_final = pd.DataFrame({'Province_State' : [], 'Country_Region' : [], 'Last_Update' : [], 'Lat' : [], 'Long_' : [],'Confirmed' : [], 'Deaths' : [], 'Recovered' : [], 'Active' : [], 'Combined_Key' : [],'Incident_Rate' : [], 'Case_Fatality_Ratio' : []})
for file in os.listdir(path):
    if file.endswith(".csv"):
        df = pd.read_csv(path + file, index_col=False)
        try:
            df_temp = df[df['Country_Region'] == "Pakistan"].copy()
            df_temp.drop(['FIPS', 'Admin2'], axis = 1, inplace = True)
            
            if(len(df_temp.columns) == 12):
                df_final = df_final.append(df_temp, True)
                df_final.drop(['Combined_Key'], axis = 1, inplace = True)
               
        except :
            continue
            

In [39]:
 df_final.drop(['Recovered', 'Active'], axis = 1, inplace = True)
df_final = df_final.dropna(subset=['Province_State'])

In [40]:
df_final.drop(['Incident_Rate', 'Incidence_Rate', 'Case-Fatality_Ratio', 'Case_Fatality_Ratio'], axis = 1, inplace = True)

In [41]:
df_final.isnull().sum()

Province_State    0
Country_Region    0
Last_Update       0
Lat               0
Long_             0
Confirmed         0
Deaths            0
dtype: int64

In [42]:
# df_temp = df_final['Case_Fatality_Ratio'].fillna(df_final['Case-Fatality_Ratio'])
# df_final.loc[df_final['Case_Fatality_Ratio'].isnull(), 'Case_Fatality_Ratio'] = df_temp

In [43]:
# df_temp = df_final['Incident_Rate'].fillna(df_final['Incidence_Rate'])
# df_final.loc[df_final['Incident_Rate'].isnull(), 'Incident_Rate'] = df_temp

In [44]:
# df_final = df_final.dropna(subset=['Incidence_Rate', 'Case-Fatality_Ratio'])

In [45]:
df_final.isnull().sum()

Province_State    0
Country_Region    0
Last_Update       0
Lat               0
Long_             0
Confirmed         0
Deaths            0
dtype: int64

In [46]:
len(df_final)

3465

In [47]:
df_final.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\All_Province (06-2020 to 10-2021).csv")

# Remaining Data Preparation For Merging

In [48]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\remaining_province_wise_data\\Pakistan_Province_summary_covid19_confirmed.csv"
df_confirmed = pd.read_csv(path, index_col=False)
df_confirmed.drop(['code'], axis = 1, inplace = True)
df_confirmed.sort_values(by=['Date'])
df_confirmed['Date'] = pd.to_datetime(df_confirmed['Date'])

confirmed_cases = df_confirmed[df_confirmed['Date'] < "2020-06-11"]
confirmed_cases.head()

,Province,Date,Confirmed cases
0,Azad Jammu and Kashmir,2020-03-10,0
1,Balochistan,2020-03-10,1
2,Gilgit-Baltistan,2020-03-10,2
3,Islamabad,2020-03-10,2
4,Khyber Pakhtunkhwa,2020-03-10,0


In [49]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\remaining_province_wise_data\\Pakistan_Province_summary_covid19_death.csv"
df_deaths = pd.read_csv(path, index_col=False)
df_deaths.drop(['code'], axis = 1, inplace = True)
df_deaths.sort_values(by=['Date'])
df_deaths['Date'] = pd.to_datetime(df_deaths['Date'])

deaths = df_deaths[df_deaths['Date'] < "2020-06-11"]
deaths.head()

,Province,Date,Deaths
0,Azad Jammu and Kashmir,2020-03-10,0
1,Balochistan,2020-03-10,0
2,Gilgit-Baltistan,2020-03-10,0
3,Islamabad,2020-03-10,0
4,Khyber Pakhtunkhwa,2020-03-10,0


In [50]:
remaining_data = confirmed_cases.merge(deaths, how='inner', on=['Province', 'Date'])
remaining_data = remaining_data.rename(columns={"Confirmed cases" : "Confirmed"})
remaining_data['Date'] = pd.to_datetime(remaining_data['Date']).dt.date
remaining_data.head()

,Province,Date,Confirmed,Deaths
0,Azad Jammu and Kashmir,2020-03-10,0,0
1,Balochistan,2020-03-10,1,0
2,Gilgit-Baltistan,2020-03-10,2,0
3,Islamabad,2020-03-10,2,0
4,Khyber Pakhtunkhwa,2020-03-10,0,0


# Merging All the Data into One File

In [51]:
df_final['Last_Update'] = pd.to_datetime(df_final['Last_Update']).dt.date


In [52]:
df_final = df_final.rename(columns={"Province_State" : "Province","Last_Update" : "Date"})

In [53]:
len(remaining_data)

651

In [54]:
df_final.tail(7)

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths
3470,Azad Jammu and Kashmir,Pakistan,2021-01-01,34.027401,73.947253,8277.0,222.0
3471,Balochistan,Pakistan,2021-01-01,28.328492,65.898403,18168.0,183.0
3472,Gilgit-Baltistan,Pakistan,2021-01-01,35.792146,74.982138,4857.0,101.0
3473,Islamabad,Pakistan,2021-01-01,33.665087,73.121219,37888.0,419.0
3474,Khyber Pakhtunkhwa,Pakistan,2021-01-01,34.485332,72.091690,58701.0,1649.0
3475,Punjab,Pakistan,2021-01-01,30.811346,72.139132,138608.0,4042.0
3476,Sindh,Pakistan,2021-01-01,26.009446,68.776807,215679.0,3560.0


In [55]:
 df_final = df_final.append(remaining_data)

In [56]:
df_final.isnull().sum()

Province            0
Country_Region    651
Date                0
Lat               651
Long_             651
Confirmed           0
Deaths              0
dtype: int64

In [57]:
df_final.loc[df_final['Country_Region'].isnull(), 'Country_Region'] = "Pakistan"

In [58]:
# df_final[df_final['Province'] == 'Sindh'][['Lat', 'Long_']]
# 26.009446, 68.776807
# df_final[df_final['Province'] == 'Punjab'][['Lat', 'Long_']]
# 30.811346, 72.139132
# df_final[df_final['Province'] == 'Gilgit-Baltistan'][['Lat', 'Long_']]
# 35.792146, 74.982138
# df_final[df_final['Province'] == 'Islamabad'][['Lat', 'Long_']]
# 33.665087, 73.121219
# df_final[df_final['Province'] == 'Khyber Pakhtunkhwa'][['Lat', 'Long_']]
# 34.485332, 72.09169
# df_final[df_final['Province'] == 'Balochistan'][['Lat', 'Long_']]
# 28.328492, 65.898403
# df_final[df_final['Province'] == 'Azad Jammu and Kashmir'][['Lat', 'Long_']]
# 34.027401, 73.947253

In [59]:
df_final.loc[df_final['Province'] == 'Sindh', 'Lat'] = 26.009446
df_final.loc[df_final['Province'] == 'Sindh', 'Long_'] = 68.776807

df_final.loc[df_final['Province'] == 'Punjab', 'Lat'] = 30.811346
df_final.loc[df_final['Province'] == 'Punjab', 'Long_'] = 72.139132

df_final.loc[df_final['Province'] == 'Gilgit-Baltistan', 'Lat'] = 35.792146
df_final.loc[df_final['Province'] == 'Gilgit-Baltistan', 'Long_'] = 74.982138

df_final.loc[df_final['Province'] == 'Islamabad', 'Lat'] = 33.665087
df_final.loc[df_final['Province'] == 'Islamabad', 'Long_'] = 73.121219

df_final.loc[df_final['Province'] == 'Khyber Pakhtunkhwa', 'Lat'] = 34.485332
df_final.loc[df_final['Province'] == 'Khyber Pakhtunkhwa', 'Long_'] = 72.09169

df_final.loc[df_final['Province'] == 'Balochistan', 'Lat'] = 28.328492
df_final.loc[df_final['Province'] == 'Balochistan', 'Long_'] = 65.898403

df_final.loc[df_final['Province'] == 'Azad Jammu and Kashmir', 'Lat'] = 34.027401
df_final.loc[df_final['Province'] == 'Azad Jammu and Kashmir', 'Long_'] = 73.947253

In [60]:
df_final.isnull().sum()

Province          0
Country_Region    0
Date              0
Lat               0
Long_             0
Confirmed         0
Deaths            0
dtype: int64

In [61]:
df_final.head()

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths
0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0
1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0
2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0
3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0
4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0


In [62]:
df_final.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\All_Province (03-2020 to 10-2021).csv")

# Working on Data From IHME

In [63]:
import pandas as pd
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\IHME_data\\all_province.csv"

In [64]:
df_All_IHME = pd.read_csv(path)
df_All_IHME.head()

,Unnamed: 0,location_id,date,version_name,location_name,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,...,icu_beds_upper,icu_beds_lower,admis_mean,admis_upper,admis_lower,all_bed_capacity,icu_bed_capacity,infection_fatality,infection_detection,infection_hospitalization
0,0,53616,2/16/2020,reference,Balochistan,19.82,212.29,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,6767,201,NaN,NaN,NaN
1,0,53618,2/16/2020,reference,Islamabad Capital Territory,23.56,199.51,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,2567,52,NaN,NaN,NaN
2,0,53620,2/16/2020,reference,Punjab,8.01,83.24,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,51809,1601,NaN,NaN,NaN
3,0,53621,2/16/2020,reference,Sindh,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,31732,848,NaN,NaN,NaN
4,0,53619,2/16/2020,reference,Khyber Pakhtunkhwa,236.38,1674.56,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,23948,643,NaN,NaN,NaN


In [65]:
df_All_IHME.drop([df_All_IHME.columns[0], "location_id", "version_name"], axis = 1, inplace = True)

In [66]:
len(df_All_IHME)

4291

In [67]:
nan_value = float("NaN")
df_All_IHME.replace("", nan_value, inplace=True)
df_All_IHME.dropna(how='any', axis=1, inplace=True)

In [68]:
df_All_IHME.isnull().sum()

date                     0
location_name            0
inf_mean                 0
inf_upper                0
inf_lower                0
seir_cumulative_mean     0
seir_cumulative_lower    0
seir_cumulative_upper    0
inf_cuml_mean            0
inf_cuml_lower           0
inf_cuml_upper           0
seir_daily_mean          0
seir_daily_lower         0
seir_daily_upper         0
population               0
mobility_mean            0
mobility_obs             0
testing_obs              0
pneumonia_mean           0
pneumonia_obs            0
mask_use_mean            0
mask_use_obs             0
hospital_beds_mean       0
hospital_beds_upper      0
hospital_beds_lower      0
icu_beds_mean            0
icu_beds_upper           0
icu_beds_lower           0
all_bed_capacity         0
icu_bed_capacity         0
dtype: int64

In [69]:
df_All_IHME = df_All_IHME.rename(columns={'date' : 'Date', 'location_name' : 'Province'})

In [70]:
df_All_IHME['Date'] = pd.to_datetime(df_All_IHME['Date']).dt.date

In [71]:
df_All_IHME.head(7)

,Date,Province,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,inf_cuml_mean,inf_cuml_lower,...,mask_use_mean,mask_use_obs,hospital_beds_mean,hospital_beds_upper,hospital_beds_lower,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity
0,2020-02-16,Balochistan,19.82,212.29,0.0,0.0,0.0,0.0,23.26,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,6767,201
1,2020-02-16,Islamabad Capital Territory,23.56,199.51,0.0,0.0,0.0,0.0,25.94,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,2567,52
2,2020-02-16,Punjab,8.01,83.24,0.0,0.0,0.0,0.0,8.34,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,51809,1601
3,2020-02-16,Sindh,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,31732,848
4,2020-02-16,Khyber Pakhtunkhwa,236.38,1674.56,0.0,0.0,0.0,0.0,268.08,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,23948,643
5,2020-02-16,Azad Jammu & Kashmir,11.22,116.64,0.0,0.0,0.0,0.0,11.89,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,2009,59
6,2020-02-16,Gilgit-Baltistan,1.82,28.12,0.0,0.0,0.0,0.0,1.82,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1423,36


In [72]:
df_All_IHME.loc[df_All_IHME['Province'] == "Azad Jammu & Kashmir", 'Province'] = "Azad Jammu and Kashmir"
df_All_IHME.loc[df_All_IHME['Province'] == "Islamabad Capital Territory", 'Province'] = "Islamabad"

In [73]:
df_All_IHME.head(7)

,Date,Province,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,inf_cuml_mean,inf_cuml_lower,...,mask_use_mean,mask_use_obs,hospital_beds_mean,hospital_beds_upper,hospital_beds_lower,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity
0,2020-02-16,Balochistan,19.82,212.29,0.0,0.0,0.0,0.0,23.26,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,6767,201
1,2020-02-16,Islamabad,23.56,199.51,0.0,0.0,0.0,0.0,25.94,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,2567,52
2,2020-02-16,Punjab,8.01,83.24,0.0,0.0,0.0,0.0,8.34,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,51809,1601
3,2020-02-16,Sindh,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,31732,848
4,2020-02-16,Khyber Pakhtunkhwa,236.38,1674.56,0.0,0.0,0.0,0.0,268.08,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,23948,643
5,2020-02-16,Azad Jammu and Kashmir,11.22,116.64,0.0,0.0,0.0,0.0,11.89,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,2009,59
6,2020-02-16,Gilgit-Baltistan,1.82,28.12,0.0,0.0,0.0,0.0,1.82,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,1423,36


In [74]:
df_All_IHME.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\IHME.csv")

# Merging All the Data from JHU and IHME

In [424]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\All_Province (03-2020 to 10-2021).csv"

In [425]:
df_JHU = pd.read_csv(path)
df_JHU.head(7)

,Unnamed: 0,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths
0,0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0
1,1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0
2,2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0
3,3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0
4,4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0
5,5,Punjab,Pakistan,2021-02-01,30.811346,72.139132,139341.0,4085.0
6,6,Sindh,Pakistan,2021-02-01,26.009446,68.776807,216632.0,3582.0


In [426]:
df_JHU['Date'] = pd.to_datetime(df_JHU['Date']).dt.date

In [427]:
df_JHU_IHME = pd.merge(left=df_JHU, right=df_All_IHME, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])

In [428]:
df_JHU_IHME.head() 

,Unnamed: 0,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,inf_mean,inf_upper,...,mask_use_mean,mask_use_obs,hospital_beds_mean,hospital_beds_upper,hospital_beds_lower,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity
0,0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0,6594.18,8468.72,...,0.4227,1,730.74,1237.83,260.98,136.10,230.55,48.61,2009,59
1,1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0,3745.17,6088.59,...,0.2686,1,672.37,1297.25,171.70,125.23,241.62,31.98,6767,201
2,2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0,168.07,284.65,...,0.3511,1,27.77,53.24,13.44,5.17,9.92,2.50,1423,36
3,3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0,1529.93,2278.70,...,0.4731,1,191.88,207.61,166.30,35.74,38.67,30.97,2567,52
4,4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0,34826.77,44617.94,...,0.3982,1,6065.40,8454.16,2630.28,1129.70,1574.61,489.90,23948,643


In [429]:
df_JHU_IHME.isnull().sum()

Unnamed: 0               0
Province                 0
Country_Region           0
Date                     0
Lat                      0
Long_                    0
Confirmed                0
Deaths                   0
inf_mean                 0
inf_upper                0
inf_lower                0
seir_cumulative_mean     0
seir_cumulative_lower    0
seir_cumulative_upper    0
inf_cuml_mean            0
inf_cuml_lower           0
inf_cuml_upper           0
seir_daily_mean          0
seir_daily_lower         0
seir_daily_upper         0
population               0
mobility_mean            0
mobility_obs             0
testing_obs              0
pneumonia_mean           0
pneumonia_obs            0
mask_use_mean            0
mask_use_obs             0
hospital_beds_mean       0
hospital_beds_upper      0
hospital_beds_lower      0
icu_beds_mean            0
icu_beds_upper           0
icu_beds_lower           0
all_bed_capacity         0
icu_bed_capacity         0
dtype: int64

In [430]:
df_JHU_IHME.rename(columns={'Unnamed: 0':'S.No'}, inplace=True )

In [431]:
df_JHU_IHME.head()

,S.No,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,inf_mean,inf_upper,...,mask_use_mean,mask_use_obs,hospital_beds_mean,hospital_beds_upper,hospital_beds_lower,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity
0,0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0,6594.18,8468.72,...,0.4227,1,730.74,1237.83,260.98,136.10,230.55,48.61,2009,59
1,1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0,3745.17,6088.59,...,0.2686,1,672.37,1297.25,171.70,125.23,241.62,31.98,6767,201
2,2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0,168.07,284.65,...,0.3511,1,27.77,53.24,13.44,5.17,9.92,2.50,1423,36
3,3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0,1529.93,2278.70,...,0.4731,1,191.88,207.61,166.30,35.74,38.67,30.97,2567,52
4,4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0,34826.77,44617.94,...,0.3982,1,6065.40,8454.16,2630.28,1129.70,1574.61,489.90,23948,643


In [432]:
df_JHU_IHME.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\JHU_IHME.csv')

# Merging Weather related Data to JHU_IHME

In [97]:
path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\JHU_IHME.csv'

In [99]:
df_JHU_IHME = pd.read_csv(path, index_col = "S.No")

In [100]:
df_JHU_IHME.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [467]:
df_JHU_IHME.head(7)

,S.No,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,inf_mean,inf_upper,...,mask_use_mean,mask_use_obs,hospital_beds_mean,hospital_beds_upper,hospital_beds_lower,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity
0,0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0,6594.18,8468.72,...,0.4227,1,730.74,1237.83,260.98,136.10,230.55,48.61,2009,59
1,1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0,3745.17,6088.59,...,0.2686,1,672.37,1297.25,171.70,125.23,241.62,31.98,6767,201
2,2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0,168.07,284.65,...,0.3511,1,27.77,53.24,13.44,5.17,9.92,2.50,1423,36
3,3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0,1529.93,2278.70,...,0.4731,1,191.88,207.61,166.30,35.74,38.67,30.97,2567,52
4,4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0,34826.77,44617.94,...,0.3982,1,6065.40,8454.16,2630.28,1129.70,1574.61,489.90,23948,643
5,5,Punjab,Pakistan,2021-02-01,30.811346,72.139132,139341.0,4085.0,137476.20,190142.67,...,0.4955,1,30433.65,46663.62,13711.03,5668.36,8691.24,2553.72,51809,1601
6,6,Sindh,Pakistan,2021-02-01,26.009446,68.776807,216632.0,3582.0,42471.79,58304.28,...,0.5406,1,15442.30,20937.55,9749.99,2876.17,3899.68,1815.97,31732,848


In [391]:
sindh_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Sindh.csv'
punjab_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Punjab.csv'
isb_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\ISB.csv'
bal_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Bal.csv'
ajk_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\AJK.csv'
kpk_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\KPK.csv'
gb_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\GB.csv'

In [394]:
df_sindh = pd.read_csv(sindh_weather_path)
df_sindh.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_punjab = pd.read_csv(punjab_weather_path)
df_punjab.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_isb = pd.read_csv(isb_weather_path)
df_isb.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_bal = pd.read_csv(bal_weather_path)
df_bal.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_ajk = pd.read_csv(ajk_weather_path)
df_ajk.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_kpk = pd.read_csv(kpk_weather_path)
df_kpk.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_gb = pd.read_csv(gb_weather_path)
df_gb.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [395]:
df_sindh.head()

,"January 1, 2020","January 2, 2020","January 3, 2020","January 4, 2020","January 5, 2020","January 6, 2020","January 7, 2020","January 8, 2020","January 9, 2020","January 10, 2020",...,"November 8, 2021","November 9, 2021","November 10, 2021","November 11, 2021","November 12, 2021","November 13, 2021","November 14, 2021","November 15, 2021",Previous 24 hours,"November 16, 2021"
0,"{'Time': '5:00 amWed, Jan 1', 'Temp': '44°F', ...","{'Time': '5:00 amWed, Jan 1', 'Temp': '44°F', ...","{'Time': '2:00 amThu, Jan 2', 'Temp': '50°F', ...","{'Time': '8:00 amFri, Jan 3', 'Temp': '52°F', ...","{'Time': '2:00 amSat, Jan 4', 'Temp': '54°F', ...","{'Time': '8:00 amSun, Jan 5', 'Temp': '57°F', ...","{'Time': '2:00 amMon, Jan 6', 'Temp': '60°F', ...","{'Time': '8:00 amTue, Jan 7', 'Temp': '50°F', ...","{'Time': '5:00 amWed, Jan 8', 'Temp': '53°F', ...","{'Time': '2:00 amThu, Jan 9', 'Temp': '52°F', ...",...,"{'Time': '2:00 amTue, Nov 9', 'Temp': '73°F', ...","{'Time': '8:00 amWed, Nov 10', 'Temp': '68°F',...","{'Time': '2:00 amThu, Nov 11', 'Temp': '69°F',...","{'Time': '2:00 amSat, Nov 13', 'Temp': '75°F',...","{'Time': '2:00 amSun, Nov 14', 'Temp': '69°F',...","{'Time': '2:00 amSun, Nov 14', 'Temp': '69°F',...","{'Time': '2:00 amMon, Nov 15', 'Temp': '65°F',...","{'Time': '2:00 amTue, Nov 16', 'Temp': '71°F',...","{'Time': '8:00 pmTue, Nov 16', 'Temp': '78°F',...","{'Time': '8:00 pmTue, Nov 16', 'Temp': '78°F',..."
1,"{'Time': '8:00 am', 'Temp': '44°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '44°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '48°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '58°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '50°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '69°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '58°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '55°F', 'Weather'...","{'Time': '8:00 am', 'Temp': '51°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '49°F', 'Weather':...",...,"{'Time': '5:00 am', 'Temp': '69°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '84°F', 'Weather'...","{'Time': '8:00 am', 'Temp': '69°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '71°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '65°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '65°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '81°F', 'Weather'...","{'Time': '8:00 am', 'Temp': '69°F', 'Weather':...","{'Time': '5:00 pm', 'Temp': '84°F', 'Weather':...","{'Time': '5:00 pm', 'Temp': '84°F', 'Weather':..."
2,"{'Time': '11:00 am', 'Temp': '50°F', 'Weather'...","{'Time': '11:00 am', 'Temp': '50°F', 'Weather'...","{'Time': '11:00 am', 'Temp': '52°F', 'Weather'...","{'Time': '5:00 pm', 'Temp': '66°F', 'Weather':...","{'Time': '5:00 pm', 'Temp': '77°F', 'Weather':...","{'Time': '5:00 pm', 'Temp': '77°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '55°F', 'Weather':...","{'Time': '2:00 pm', 'Temp': '60°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '60°F', 'Weather'...","{'Time': '11:00 am', 'Temp': '60°F', 'Weather'...",...,"{'Time': '2:00 pm', 'Temp': '91°F', 'Weather':...","{'Time': '11:00 pm', 'Temp': '72°F', 'Weather'...","{'Time': '11:00 am', 'Temp': '82°F', 'Weather'...","{'Time': '2:00 pm', 'Temp': '88°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '64°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '64°F', 'Weather':...","{'Time': '2:00 pm', 'Temp': '88°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '77°F', 'Weather'...","{'Time': '2:00 pm', 'Temp': '85°F', 'Weather':...","{'Time': '2:00 pm', 'Temp': '85°F', 'Weather':..."
3,"{'Time': '2:00 pm', 'Temp': '55°F', 'Weather':...","{'Time': '2:00 pm', 'Temp': '55°F', 'Weather':...","{'Time': '2:00 pm', 'Temp': '60°F', 'Weather':...","{'Time': '8:00 pm', 'Temp': '62°F', 'Weather':...","{'Time': '8:00 pm', 'Temp': '65°F', 'Weather':...","{'Time': '8:00 pm', 'Temp': '71°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '61°F', 'Weather'...","{'Time': '5:00 pm', 'Temp': '62°F', 'Weather':...","{'Time': '8:00 pm', 'Temp': '59°F', 'Weather':...","{'Time': 

In [396]:
ast.literal_eval(df_sindh['January 1, 2020'][0])

{'Time': '5:00 amWed, Jan 1',
 'Temp': '44°F',
 'Weather': 'Fog.',
 'Wind': '9 mph',
 'Humidity': '↑',
 'Barometer': '86%',
 'Visibility': '30.06 "Hg'}

In [397]:
ast.literal_eval(df_sindh['January 1, 2020'][3])['Weather']

'Partly sunny.'

In [439]:
sindh_final_dict = {}
for i in df_sindh.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_sindh[i])):
        try:
            tTemp += float(ast.literal_eval(df_sindh[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_sindh[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_sindh[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_sindh[i][j])['Visibility'][:5])
            
        except:
            continue
    sindh_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_sindh[i][0])['Weather']}
    

In [399]:
punjab_final_dict = {}
for i in df_punjab.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_punjab[i])):
        try:
            tTemp += float(ast.literal_eval(df_punjab[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_punjab[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_punjab[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_punjab[i][j])['Visibility'][:5])
            
        except:
            continue
    punjab_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_punjab[i][0])['Weather']}
    

In [400]:
isb_final_dict = {}
for i in df_isb.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_isb[i])):
        try:
            tTemp += float(ast.literal_eval(df_isb[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_isb[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_isb[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_isb[i][j])['Visibility'][:5])
            
        except:
            continue
    isb_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_isb[i][0])['Weather']}
    

In [401]:
bal_final_dict = {}
for i in df_bal.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_bal[i])):
        try:
            tTemp += float(ast.literal_eval(df_bal[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_bal[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_bal[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_bal[i][j])['Visibility'][:5])
            
        except:
            continue
    bal_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_bal[i][0])['Weather']}
    

In [402]:
ajk_final_dict = {}
for i in df_ajk.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_ajk[i])):
        try:
            tTemp += float(ast.literal_eval(df_ajk[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_ajk[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_ajk[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_ajk[i][j])['Visibility'][:5])
            
        except:
            continue
    ajk_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_ajk[i][0])['Weather']}
    

In [403]:
kpk_final_dict = {}
for i in df_kpk.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_kpk[i])):
        try:
            tTemp += float(ast.literal_eval(df_kpk[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_kpk[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_kpk[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_kpk[i][j])['Visibility'][:5])
            
        except:
            continue
    kpk_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_kpk[i][0])['Weather']}
    

In [404]:
gb_final_dict = {}
for i in df_gb.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_gb[i])):
        try:
            tTemp += float(ast.literal_eval(df_gb[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_gb[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_gb[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_gb[i][j])['Visibility'][:5])
            
        except:
            continue
    gb_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_gb[i][0])['Weather']}
    

In [440]:
pd.DataFrame(sindh_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Sindh_Final_Weather.csv')

In [418]:
pd.DataFrame(punjab_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Punjab_Final_Weather.csv')

In [419]:
pd.DataFrame(isb_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\ISB_Final_Weather.csv')

In [420]:
pd.DataFrame(kpk_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\KPK_Final_Weather.csv')

In [421]:
pd.DataFrame(ajk_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\AJK_Final_Weather.csv')

In [422]:
pd.DataFrame(bal_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\BAL_Final_Weather.csv')

In [423]:
pd.DataFrame(gb_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\GB_Final_Weather.csv')

In [434]:
# df_JHU_IHME

In [468]:
dfw_sindh = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Sindh_Final_Weather.csv')
dfw_sindh["Province"] = "Sindh"
dfw_punjab = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Punjab_Final_Weather.csv')
dfw_punjab["Province"] = "Punjab"
dfw_ajk = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\AJK_Final_Weather.csv')
dfw_ajk["Province"] = "Azad Jammu and Kashmir"
dfw_kpk = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\KPK_Final_Weather.csv')
dfw_kpk["Province"] = "Khyber Pakhtunkhwa"
dfw_bal = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\BAL_Final_Weather.csv')
dfw_bal["Province"] = "Balochistan"
dfw_gb = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\ISB_Final_Weather.csv')
dfw_gb["Province"] = "Gilgit-Baltistan"
dfw_isb = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Sindh_Final_Weather.csv')
dfw_isb["Province"] = "Islamabad"
# dfw_sindh.rename(columns={'Unnamed: 0':'Date'}, inplace=True )

In [476]:
dfw_all_provinces = pd.concat([dfw_sindh, dfw_punjab, dfw_ajk, dfw_kpk, dfw_bal, dfw_gb, dfw_isb])
dfw_all_provinces.rename(columns={'Unnamed: 0':'Date'}, inplace=True )
dfw_all_provinces[dfw_all_provinces['Date'] == "Previous 24 hours"] = ""

In [477]:
dfw_all_provinces.head()

,Date,Temp,Wind,Barometer,Visibility,Weather,Province
0,"January 1, 2020",43.43,16.43,60.57,25.81,Fog.,Sindh
1,"January 2, 2020",43.43,16.43,60.57,25.81,Fog.,Sindh
2,"January 3, 2020",30.0,9.57,39.86,17.23,Clear.,Sindh
3,"January 4, 2020",42.29,6.14,34.29,17.22,Scattered clouds.,Sindh
4,"January 5, 2020",44.14,0.0,0.0,0.0,Clear.,Sindh


In [478]:
from datetime import datetime
dfw_all_provinces['Date'] = pd.to_datetime(dfw_all_provinces['Date'])

In [479]:
df_JHU_IHME['Date'] = pd.to_datetime(df_JHU_IHME['Date'])

In [480]:
df_JHU_IHME_TD = pd.merge(left=df_JHU_IHME, right=dfw_all_provinces, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])

In [484]:
df_JHU_IHME_TD.head()

,S.No,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,inf_mean,inf_upper,...,icu_beds_mean,icu_beds_upper,icu_beds_lower,all_bed_capacity,icu_bed_capacity,Temp,Wind,Barometer,Visibility,Weather
0,0,Azad Jammu and Kashmir,Pakistan,2021-02-01,34.027401,73.947253,8303.0,224.0,6594.18,8468.72,...,136.10,230.55,48.61,2009,59,47.14,2.14,42.0,0.0,Passing clouds.
1,1,Balochistan,Pakistan,2021-02-01,28.328492,65.898403,18181.0,184.0,3745.17,6088.59,...,125.23,241.62,31.98,6767,201,46.29,7.29,25.71,0.0,Clear.
2,2,Gilgit-Baltistan,Pakistan,2021-02-01,35.792146,74.982138,4862.0,101.0,168.07,284.65,...,5.17,9.92,2.50,1423,36,49.57,4.29,33.71,21.53,Passing clouds.
3,3,Islamabad,Pakistan,2021-02-01,33.665087,73.121219,38020.0,421.0,1529.93,2278.70,...,35.74,38.67,30.97,2567,52,49.43,2.71,10.43,8.58,Sunny.
4,4,Khyber Pakhtunkhwa,Pakistan,2021-02-01,34.485332,72.091690,59023.0,1661.0,34826.77,44617.94,...,1129.70,1574.61,489.90,23948,643,42.73,3.52,30.5,19.86,Passing clouds.


In [483]:
df_JHU_IHME_TD.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\JHU_IHME_TD.csv')

# Preparing Vaccination Data

In [269]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\vaccinations_data\\vaccinations.csv"

In [301]:
df_vaccinations = pd.read_csv(path)

In [302]:
df_vac_pak = df_vaccinations[df_vaccinations['location'] == "Pakistan"]
df_vac_pak.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
38399,Pakistan,PAK,2021-02-02,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
38400,Pakistan,PAK,2021-02-03,NaN,NaN,NaN,NaN,NaN,3404.0,NaN,NaN,NaN,NaN,15.0
38401,Pakistan,PAK,2021-02-04,NaN,NaN,NaN,NaN,NaN,3404.0,NaN,NaN,NaN,NaN,15.0
38402,Pakistan,PAK,2021-02-05,NaN,NaN,NaN,NaN,NaN,3404.0,NaN,NaN,NaN,NaN,15.0
38403,Pakistan,PAK,2021-02-06,NaN,NaN,NaN,NaN,NaN,3404.0,NaN,NaN,NaN,NaN,15.0


In [303]:
df_vac_pak = df_vac_pak[['date', 'daily_vaccinations', 'daily_vaccinations_per_million']]
df_vac_pak = df_vac_pak.dropna(subset=['daily_vaccinations', 'daily_vaccinations_per_million'])
df_vac_pak.head()

,date,daily_vaccinations,daily_vaccinations_per_million
38400,2021-02-03,3404.0,15.0
38401,2021-02-04,3404.0,15.0
38402,2021-02-05,3404.0,15.0
38403,2021-02-06,3404.0,15.0
38404,2021-02-07,3404.0,15.0


# Merging Vaccination Data to JHU_IHME

In [30]:
path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\JHU_IHME.csv'
df_JHU_IHME = pd.read_csv(path)

In [ ]:
df_JHU_IHME_Vac = pd.merge(left=df_JHU_IHME, right=df_vac_pak, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])